In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the saved model
model = tf.keras.models.load_model("Model_V3_7525.h5")

# Define class names
class_names = ['Ashy crowned sparrow lark', 'Asian Openbill', 'Black-headed ibis', 'Crow', 'Eurasian Coot', 
               'Indian Roller', 'Large-billed Crow', 'Little Cormorant', 'Paddyfield pipit', 'Painted Stork', 
               'Red-wattled lapwing', 'Spot-billed Pelician', 'White-breasted Waterhen', 'Yellow wattled lapwing']

# Extended class list for internal use (includes 'None')
class_names_extended = class_names + ['None']

# Preprocess image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Bayesian update function
def bayesian_update(prior_probs, likelihoods):
    numerators = likelihoods * prior_probs
    denominator = np.sum(numerators)
    if denominator == 0:
        return np.zeros_like(numerators)
    return numerators / denominator

# Image path
input_image_path = "E:/CAPSTONE/Bird_Species_Classification/Models/FINAL MODELS/EffiecientB7/SDP_Project/Images for Testing/images (4).jpeg"

# Simulated user input
user_selected_counts = {
    'Crow': 3,
    'Large-billed Crow': 0 
}

if list(user_selected_counts.keys()) == ['None']:
    print("❌ No bird in dataset. Skipping prediction and Bayesian update.")
else:
    img_array = preprocess_image(input_image_path)
    
    # Model prediction
    predictions = model.predict(img_array)
    prior_probs = predictions[0]

    # Add zero prior for 'None'
    prior_probs = np.append(prior_probs, 0.0)

    # Print initial model prediction
    predicted_class_index = np.argmax(prior_probs)
    print(f"\n🧠 Initial Prediction: {class_names_extended[predicted_class_index]}")

    # Print initial probabilities
    print("\n--- Initial Class Probabilities ---")
    for i, prob in enumerate(prior_probs):
        print(f"{class_names_extended[i]:30}: {prob:.4f}")

    # Compute likelihoods from user input
    likelihoods = np.zeros_like(prior_probs)
    total_selected = sum(user_selected_counts.values())

    for class_name, count in user_selected_counts.items():
        if class_name in class_names_extended:
            index = class_names_extended.index(class_name)
            likelihoods[index] = count / total_selected

    # Apply Bayesian update
    posterior_probs = bayesian_update(prior_probs, likelihoods)
    final_class_index = np.argmax(posterior_probs)
    final_class_name = class_names_extended[final_class_index]

    # Show updated prediction
    if final_class_name == 'None':
        print("\n❌ Bird not in our dataset.")
    else:
        print(f"\n🎯 After Bayesian Update: {final_class_name}")

    # Print comparison
    print("\n--- Class Probabilities Comparison ---")
    print(f"{'Class Name':30} | {'Before':>8} | {'After':>8}")
    print("-" * 55)
    for i in range(len(class_names_extended)):
        print(f"{class_names_extended[i]:30} | {prior_probs[i]:8.4f} | {posterior_probs[i]:8.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step

🧠 Initial Prediction: Large-billed Crow

--- Initial Class Probabilities ---
Ashy crowned sparrow lark     : 0.0072
Asian Openbill                : 0.0009
Black-headed ibis             : 0.0007
Crow                          : 0.1087
Eurasian Coot                 : 0.0000
Indian Roller                 : 0.0102
Large-billed Crow             : 0.4394
Little Cormorant              : 0.0019
Paddyfield pipit              : 0.0393
Painted Stork                 : 0.0000
Red-wattled lapwing           : 0.0109
Spot-billed Pelician          : 0.0001
White-breasted Waterhen       : 0.0013
Yellow wattled lapwing        : 0.3795
None                          : 0.0000

🎯 After Bayesian Update: Crow

--- Class Probabilities Comparison ---
Class Name                     |   Before |    After
-------------------------------------------------------
Ashy crowned sparrow lark      |   0.0072 |   0.0000
Asian Openbill                 |   0.0009 |   0.0000
Black-headed ib